# Importing The Neccesary Libraries

In [60]:
import nltk
import json
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import warnings
warnings.filterwarnings('ignore')

# Creating Query-Based Intents for the Dataset

In [61]:
with open("C:/Users/HP/Downloads/intents_file.txt","r") as file:
    file_content = file.read()
    intents = dict(eval(file_content))
    print("before appending",len(intents["intents"]))

conn = sqlite3.connect(r"C:/Users/HP/Downloads/chatbot_db.db")
cursor = conn.cursor()
cursor.execute("SELECT tag, patterns, responses FROM student_response")
records = cursor.fetchall()
if 50 >= 50:
        for record in records:
            tag = record[0]
            patterns = [record[1]]
            responses = [record[2]]
            intent = {
                "tag": tag,
                "patterns": patterns,
                "responses": responses
            }
            intents["intents"].append(intent)
            with open("C:/Users/HP/Downloads/intents_file.txt","w") as file:
                file.write(str(intents))
            print("after appending",len(intents["intents"]))

before appending 273
after appending 274
after appending 275
after appending 276
after appending 277
after appending 278
after appending 279
after appending 280
after appending 281
after appending 282
after appending 283
after appending 284
after appending 285
after appending 286
after appending 287


### Downloading NLP Package

In [62]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Processing Data and Synonym Augmentation for Intent Classification

In [63]:
stemmer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

1096 documents
223 classes ['About Canteen', 'Greeting of Hi', 'Greetings of Hi', 'academic_advice_services', 'academic_advising', 'academic_advising_system', 'academic_advisory_system', 'academic_credit_transfer', 'academic_credits_transfer', 'academic_system', 'academics_majors', 'academics_reputation', 'accessibility', 'admission', 'admission_requirements_international_students', 'admissions_interview_role', 'alumni_association', 'alumni_network', 'alumni_network_activity', 'athletic_scholarships', 'average_class_size_intro', 'average_class_size_upper_division', 'average_federal_plus_loan_debt', 'average_financial_aid_package', 'average_loan_debt', 'average_merit_award', 'bond_rating', 'campus_diversity', 'campus_facilities', 'campus_housing_percentage', 'campus_navigation', 'campus_safety_security', 'campus_wifi', 'canteen_items', 'career_planning_services', 'career_services', 'career_services_access', 'career_services_alumni', 'career_services_for_alumni', 'casual_greeting', 'clas

In [64]:
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)


def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i + 1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences


# Augment the training data using synonym replacement
augmented_data = []
limit_per_tag = 100

for i, doc in enumerate(training):
    bag, output_row = doc
    tokens = [words[j] for j in range(len(words)) if bag[j] == 1]
    augmented_sentences = synonym_replacement(tokens, limit_per_tag)
    for augmented_sentence in augmented_sentences:
        augmented_bag = [1 if augmented_sentence.find(word) >= 0 else 0 for word in words]
        augmented_data.append([augmented_bag, output_row])



print("training: \n",training)
print("augmented: \n",augmented_data)

training_array = np.array([(doc[0], doc[1]) for doc in training], dtype=object)
augmented_array = np.array([(doc[0], doc[1]) for doc in augmented_data], dtype=object)

combined_data = np.concatenate((training_array, augmented_array), axis=0)
np.random.shuffle(combined_data)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Splitting the Dataset

In [65]:
from sklearn.model_selection import train_test_split


def separate_data_by_tags(data):
    data_by_tags = {}
    for d in data:
        tag = tuple(d[1])
        if tag not in data_by_tags:
            data_by_tags[tag] = []
        data_by_tags[tag].append(d)
    return data_by_tags.values()


separated_data = separate_data_by_tags(combined_data)

# Lists to store training and testing data
training_data = []
testing_data = []

# Split each tag's data into training and testing sets
for tag_data in separated_data:
    train_data, test_data = train_test_split(tag_data, test_size=0.2, random_state=42)
    training_data.extend(train_data)
    testing_data.extend(test_data)


random.shuffle(training_data)
random.shuffle(testing_data)

# Convert training and testing data back to np.array
train_x = np.array([d[0] for d in training_data])
train_y = np.array([d[1] for d in training_data])
test_x = np.array([d[0] for d in testing_data])
test_y = np.array([d[1] for d in testing_data])

# Training The Model

In [66]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        output = self.softmax(x)
        return output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    true_labels = torch.argmax(targets, dim=1)
    correct = (predicted_labels == true_labels).sum().item()
    total = targets.size(0)
    return correct / total

def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = len(test_loader)

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_accuracy += accuracy(outputs, targets) * inputs.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    average_accuracy = total_accuracy / len(test_loader.dataset)
    return average_loss, average_accuracy

In [67]:
# Create DataLoader for training and testing data
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

batch_size = 64
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model, loss function, and optimizer
input_size = len(train_x[0])
hidden_size = 8
output_size = len(train_y[0])
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model and evaluate on the testing set
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, targets) * inputs.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)

    # Print training loss and accuracy for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")

    # Evaluate on the testing set
    test_loss, test_accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch [1/50], Training Loss: 0.0201, Training Accuracy: 0.2357
Epoch [1/50], Testing Loss: 0.0130, Testing Accuracy: 0.4812
Epoch [2/50], Training Loss: 0.0103, Training Accuracy: 0.6013
Epoch [2/50], Testing Loss: 0.0083, Testing Accuracy: 0.6817
Epoch [3/50], Training Loss: 0.0072, Training Accuracy: 0.7322
Epoch [3/50], Testing Loss: 0.0061, Testing Accuracy: 0.7784
Epoch [4/50], Training Loss: 0.0052, Training Accuracy: 0.8172
Epoch [4/50], Testing Loss: 0.0044, Testing Accuracy: 0.8491
Epoch [5/50], Training Loss: 0.0039, Training Accuracy: 0.8638
Epoch [5/50], Testing Loss: 0.0035, Testing Accuracy: 0.8804
Epoch [6/50], Training Loss: 0.0032, Training Accuracy: 0.8876
Epoch [6/50], Testing Loss: 0.0030, Testing Accuracy: 0.8901
Epoch [7/50], Training Loss: 0.0028, Training Accuracy: 0.9017
Epoch [7/50], Testing Loss: 0.0027, Testing Accuracy: 0.9055
Epoch [8/50], Training Loss: 0.0025, Training Accuracy: 0.9118
Epoch [8/50], Testing Loss: 0.0024, Testing Accuracy: 0.9109
Epoch [9

# Model Inference

In [68]:
def load_model(model_path, input_size, hidden_size, output_size):
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to preprocess the input sentence
def preprocess_sentence(sentence, words):
    sentence_words = sentence.lower().split()
    sentence_words = [word for word in sentence_words if word in words]
    return sentence_words

# Function to convert the preprocessed sentence into a feature vector
def sentence_to_features(sentence_words, words):
    features = [1 if word in sentence_words else 0 for word in words]
    return torch.tensor(features).float().unsqueeze(0)

# Function to generate a response using the trained model
def generate_response(sentence, model, words, classes):
    sentence_words = preprocess_sentence(sentence, words)
    if len(sentence_words) == 0:
        return "I'm sorry, but I don't understand. Can you please rephrase or provide more information?"

    features = sentence_to_features(sentence_words, words)
    with torch.no_grad():
        outputs = model(features)

    probabilities, predicted_class = torch.max(outputs, dim=1)
    confidence = probabilities.item()
    predicted_tag = classes[predicted_class.item()]

    if confidence > 0.5:
        for intent in intents['intents']:
            if intent['tag'] == predicted_tag:
                return random.choice(intent['responses'])

    return "I'm sorry, but I'm not sure how to respond to that."

In [69]:
model_path = 'model.pth'
input_size = len(words)
hidden_size = 8
output_size = len(classes)
model = load_model(model_path, input_size, hidden_size, output_size)

# Test the chatbot response
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input, model, words, classes)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.


>  hello


Hello!


>  quit


In [79]:
import tkinter as tk
from tkinter import scrolledtext, simpledialog, messagebox

def send_message():
    user_input = input_box.get("1.0", tk.END).strip()
    input_box.delete("1.0", tk.END)
    if user_input.lower() == 'quit':
        root.destroy()
    else:
        response = generate_response(user_input, model, words, classes)
        chat_history.config(state=tk.NORMAL)  # Allow modification to insert the response
        chat_history.insert(tk.END, f"User: {user_input}\n", "user")
        chat_history.insert(tk.END, f"Chatbot: {response}\n\n", "bot")
        chat_history.see(tk.END)
        chat_history.config(state=tk.DISABLED)  # Disable modification after insertion
        if response == "I'm sorry, but I'm not sure how to respond to that." or response == "I'm sorry, but I don't understand. Can you please rephrase or provide more information?":
            contribute_response_dialog(user_input)

def contribute_response_dialog(user_input):
    response = messagebox.askyesno("Contribute Response", "Contribute your response to train the bot for this question?")
    if response:
        contribution_window = tk.Toplevel(root)
        contribution_window.title("Contribute Response")

        # Labels and entry fields for user input
        tk.Label(contribution_window, text="Roll Number:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
        tk.Label(contribution_window, text="Tag/Category:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
        tk.Label(contribution_window, text="Question:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
        tk.Label(contribution_window, text="Response:").grid(row=3, column=0, padx=5, pady=5, sticky="e")

        roll_number_entry = tk.Entry(contribution_window)
        roll_number_entry.grid(row=0, column=1, padx=5, pady=5, sticky="w")
        tag_category_entry = tk.Entry(contribution_window)
        tag_category_entry.grid(row=1, column=1, padx=5, pady=5, sticky="w")
        question_entry = tk.Entry(contribution_window)
        question_entry.grid(row=2, column=1, padx=5, pady=5, sticky="w")
        question_entry.insert(tk.END, user_input)  # Display the question in the entry field
        response_entry = tk.Entry(contribution_window)
        response_entry.grid(row=3, column=1, padx=5, pady=5, sticky="w")

        # Function to insert contribution into the database
        def insert_contribution():
            roll_number = roll_number_entry.get()
            tag_category = tag_category_entry.get()
            question = question_entry.get()
            response = response_entry.get()

            # Pattern generated based on user_input and bot's response
            patterns = f"User: {user_input}"

            try:
                conn = sqlite3.connect(r"C:/Users/HP/Downloads/chatbot_db.db")
                cursor = conn.cursor()
                cursor.execute("INSERT INTO student_response (tag, patterns, responses, roll_number) VALUES (?, ?, ?, ?)",
                               (tag_category, patterns, response, roll_number))
                conn.commit()
                conn.close()
                messagebox.showinfo("Contribution", "Thanks for your contribution! This will improve the bot's knowledge.")
                contribution_window.destroy()
                continue_chatting()
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {e}")

        # Button to submit the contribution
        submit_button = tk.Button(contribution_window, text="Submit", command=insert_contribution)
        submit_button.grid(row=4, column=0, columnspan=2, padx=5, pady=10)
    else:
        continue_chatting()

def continue_chatting():
    chat_history.config(state=tk.DISABLED)  # Disable modification after insertion

# Initialize the tkinter window
root = tk.Tk()
root.title("Chatbot")

# Set the window size and position
window_width = 600  # Adjust as needed
window_height = 600  # Adjust as needed
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_position = (screen_width - window_width) // 2
y_position = (screen_height - window_height) // 2
root.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")

# Load and display the image
image_path = "C:/Users/HP/OneDrive/Pictures/Screenshots/Screenshot 2024-03-23 204802.png"
img = tk.PhotoImage(file=image_path)
image_label = tk.Label(root, image=img)
image_label.configure(width=400,height=200)
image_label.pack()

# Create a label for the text below the image
text_label = tk.Label(root, text="L    B    R    C    E", font=("Helvetica", 12,"bold"))
text_label.pack()

# Create a frame for the chat interface with a white background
chat_frame = tk.Frame(root, bg='brown')
chat_frame.pack(fill=tk.BOTH, expand=True)

# Create a scrolled text widget for chat history
chat_history = scrolledtext.ScrolledText(chat_frame, width=50, height=15, bg='#25D366', fg='black', wrap=tk.WORD, font=("Helvetica", 12))
chat_history.pack(fill=tk.BOTH, expand=True)
chat_history.tag_config("user", foreground='black', background='yellow', justify=tk.RIGHT)
chat_history.tag_config("bot", foreground='white', background='#00703c', justify=tk.LEFT)
chat_history.config(state=tk.DISABLED)  # Make chat history read-only

# Display initial message from bot
initial_message = "Greetings! How can I help you today?"
chat_history.config(state=tk.NORMAL)  # Allow modification to insert the message
chat_history.insert(tk.END, f"Chatbot: {initial_message}\n\n", "bot")
chat_history.config(state=tk.DISABLED)  # Disable modification after insertion

# Create a text box for user input
input_box = tk.Text(chat_frame, width=40, height=3, font=("Helvetica", 12))
input_box.pack(side=tk.LEFT, padx=10, pady=10, fill=tk.X, expand=True)

# Create a button to send user input with round shape
send_button = tk.Button(chat_frame, text="Send", width=10, command=send_message, bg='#9acd32', fg='white', font=("Helvetica", 12, "bold"), relief=tk.RAISED, bd=3)
send_button.pack(side=tk.LEFT, padx=10, pady=10)

# Start the tkinter event loop
root.mainloop()


In [71]:
import sqlite3

def print_table_schema(database_path, table_name):
    try:
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        print(f"Schema for table '{table_name}':")
        for column in columns:
            column_name = column[1]
            data_type = column[2]
            nullable = "NOT NULL" if column[3] else "NULL"
            print(f"{column_name} {data_type} {nullable}")
    except sqlite3.Error as e:
        print("Error:", e)
    finally:
        if conn:
            conn.close()

# Example usage
database_path = r"C:/Users/HP/Downloads/chatbot_db.db"
table_name = "student_response"
print_table_schema(database_path, table_name)


Schema for table 'student_response':
tag VARCHAR(100) NULL
patterns VARCHAR(100) NULL
responses VARCHAR(100) NULL
roll_number VARCHAR(100) NULL


In [72]:
import sqlite3

def print_table_records(database_path, table_name):
    try:
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table_name}")
        records = cursor.fetchall()
        
        print(f"All records in table '{table_name}':")
        for record in records:
            print(record)
    except sqlite3.Error as e:
        print("Error:", e)
    finally:
        if conn:
            conn.close()

# Example usage
database_path = r"C:/Users/HP/Downloads/chatbot_db.db"
table_name = "student_response"
print_table_records(database_path, table_name)


All records in table 'student_response':
('greeting', 'Hi', 'Hello!', None)
('greeting', 'Hello', 'Good to see you!', None)
('farewell', 'Goodbye', 'Sad to see you go :(', None)
('farewell', 'Bye', 'Goodbye!', None)
('creator', 'Who created you?', 'I was created by Madhu Appala Narasimha Golthi.', None)
('creator', 'Who made you?', 'I was created by Madhu Appala Narasimha Golthi.', None)
('identity', 'What is your name?', "You can call me LBRCE-BOT. I'm a Chatbot.", None)
('identity', 'Who are you?', "You can call me LBRCE-BOT. I'm a Chatbot.", None)
('communication', 'How can I communicate with the college?', 'You can communicate with the college through our official email: lbcemym@lbrce.ac.in, helpline number: [7386349999, 9912030759],[08659 - 222933, 222934, 223936, 223937]', None)
('communication', 'What are the communication channels for students and parents?', 'You can communicate with the college through our official email: lbcemym@lbrce.ac.in, helpline number: [7386349999, 9912